In [12]:
import pandas as pd
import numpy as np
import json
import re
import time
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

%matplotlib inline
import csv

In [13]:
labels = pd.read_csv("categories_string.csv")
train_label= pd.read_csv('train_label.csv')
with open('train.json') as f:
   train = json.load(f)
with open('test.json') as h:
   test = json.load(h)

train_df = pd.DataFrame(train)
test_df = pd.DataFrame(test)
dic = labels.to_dict()
dic = dic["0"]
train_label['jobtitle'] = train_label['Category']
train_label = train_label.replace({"jobtitle": dic})
comb_data = pd.merge(train_df,train_label,how = "outer", on = 'Id' )
comb_data

,Id,description,gender,Category,jobtitle
0,0,She is also a Ronald D. Asmus Policy Entrepre...,F,19,professor
1,1,He is a member of the AICPA and WICPA. Brent ...,M,9,accountant
2,2,Dr. Aster has held teaching and research posi...,M,19,professor
3,3,He runs a boutique design studio attending cl...,M,24,architect
4,4,"He focuses on cloud security, identity and ac...",M,24,architect
...,...,...,...,...,...
217192,217192,A member of the UWA Cultural Collections Boar...,M,19,professor
217193,217193,Kelly has worked globally leading teams of co...,F,22,psychologist
217194,217194,He's the lead author of a recent study that f...,M,19,professor
217195,217195,She specializes in the theoretical and pedago...,F,19,professor


In [14]:
sentences = comb_data['description'].tolist()
len(sentences)
sentences_as_one_string =" ".join(sentences)

In [15]:
comb_data['description'].apply(lambda x: len(x.split(' '))).sum()

13504784

In [16]:
def print_train(index):
    example =comb_data[comb_data.index == index][['description', 'Id']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Tag:', example[1])

In [17]:
print_train(10)

 An elder in the United Methodist Church, she writes and lectures widely about African-American religion and is treasurer of the Society for the Study of Black Religion. She is an expert on women in the civil rights movement, and she wrote Witnessing and Testifying: Black Women, Religion, and Civil Rights (Augsburg Fortress, 2003). Contact 404-270-5527, RRoss@spelman.edu.
Tag: 10


In [18]:
sentences_test = test_df['description'].tolist()
len(sentences_test)
sentences_test_as_one_string =" ".join(sentences)

In [19]:
def print_test(Id):
    example =test_df[test_df.Id == Id][['description', 'Id']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Tag:', example[1])

In [20]:
print_test(84)

 He is coauthor with Robert N. Brandon of Biology’s First Law: The Tendency for Diversity and Complexity to Increase in Evolutionary Systems (University of Chicago Press, 2010) and is coauthor with Alex Rosenberg of Philosophy of Biology: A Contemporary Introduction (Routledge, 2008).» Post Comment
Tag: 84


In [21]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

In [22]:
%%time
comb_data['cleaned_Data'] = comb_data['description'].apply(clean_text)

Wall time: 32.8 s


In [23]:
print_train(10)

 An elder in the United Methodist Church, she writes and lectures widely about African-American religion and is treasurer of the Society for the Study of Black Religion. She is an expert on women in the civil rights movement, and she wrote Witnessing and Testifying: Black Women, Religion, and Civil Rights (Augsburg Fortress, 2003). Contact 404-270-5527, RRoss@spelman.edu.
Tag: 10


In [24]:
comb_data

,Id,description,gender,Category,jobtitle,cleaned_Data
0,0,She is also a Ronald D. Asmus Policy Entrepre...,F,19,professor,also ronald asmus policy entrepreneur fellow g...
1,1,He is a member of the AICPA and WICPA. Brent ...,M,9,accountant,member aicpa wicpa brent graduated university ...
2,2,Dr. Aster has held teaching and research posi...,M,19,professor,dr aster held teaching research positions ben ...
3,3,He runs a boutique design studio attending cl...,M,24,architect,runs boutique design studio attending clients ...
4,4,"He focuses on cloud security, identity and ac...",M,24,architect,focuses cloud security identity access managem...
...,...,...,...,...,...,...
217192,217192,A member of the UWA Cultural Collections Boar...,M,19,professor,member uwa cultural collections board gary chi...
217193,217193,Kelly has worked globally leading teams of co...,F,22,psychologist,kelly worked globally leading teams consultant...
217194,217194,He's the lead author of a recent study that f...,M,19,professor,hes lead author recent study found hospitaliza...
217195,217195,She specializes in the theoretical and pedago...,F,19,professor,specializes theoretical pedagogical foundation...


In [31]:
df = pd.DataFrame(comb_data)

print (df)

            Id                                        description gender  \
0            0   She is also a Ronald D. Asmus Policy Entrepre...      F   
1            1   He is a member of the AICPA and WICPA. Brent ...      M   
2            2   Dr. Aster has held teaching and research posi...      M   
3            3   He runs a boutique design studio attending cl...      M   
4            4   He focuses on cloud security, identity and ac...      M   
...        ...                                                ...    ...   
217192  217192   A member of the UWA Cultural Collections Boar...      M   
217193  217193   Kelly has worked globally leading teams of co...      F   
217194  217194   He's the lead author of a recent study that f...      M   
217195  217195   She specializes in the theoretical and pedago...      F   
217196  217196   Since she was 10 years old she has become a m...      F   

        Category      jobtitle  \
0             19     professor   
1              9   

In [32]:
df.to_csv (r'PreprocessedData.csv', index = False, header=True)

print (df)

            Id                                        description gender  \
0            0   She is also a Ronald D. Asmus Policy Entrepre...      F   
1            1   He is a member of the AICPA and WICPA. Brent ...      M   
2            2   Dr. Aster has held teaching and research posi...      M   
3            3   He runs a boutique design studio attending cl...      M   
4            4   He focuses on cloud security, identity and ac...      M   
...        ...                                                ...    ...   
217192  217192   A member of the UWA Cultural Collections Boar...      M   
217193  217193   Kelly has worked globally leading teams of co...      F   
217194  217194   He's the lead author of a recent study that f...      M   
217195  217195   She specializes in the theoretical and pedago...      F   
217196  217196   Since she was 10 years old she has become a m...      F   

        Category      jobtitle  \
0             19     professor   
1              9   

In [33]:
%%time
test_df['cleaned_Data'] = test_df['description'].apply(clean_text)

Wall time: 8.17 s


In [34]:
test_df.to_csv (r'PreprocessedDataTest.csv', index = False, header=True)

In [49]:
print_test(84)

coauthor robert n brandon biologys first law tendency diversity complexity increase evolutionary systems university chicago press 2010 coauthor alex rosenberg philosophy biology contemporary introduction routledge 2008 post comment
Tag: 84


In [61]:
X_test = test_df.description

In [50]:
X = comb_data.description
y = comb_data.Category

print(X.shape)
print(y.shape)

(217197,)
(217197,)


In [51]:
X_train, X_valid, y_train ,y_valid = train_test_split(X, y, test_size=0.25, random_state = 1)

In [56]:
svm = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LinearSVC()),
                     ])

In [57]:
svm.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LinearSVC())])

In [58]:
predicted = svm.predict(X_valid)

In [59]:
print(metrics.classification_report(y_valid, predicted))

              precision    recall  f1-score   support

           0       0.63      0.57      0.60       361
           1       0.77      0.71      0.74      1015
           2       0.74      0.67      0.70       248
           3       0.57      0.53      0.55      2285
           4       0.78      0.70      0.74       194
           5       0.76      0.80      0.78      1117
           6       0.68      0.73      0.70      3114
           7       0.74      0.64      0.69       211
           8       0.79      0.71      0.75      1656
           9       0.82      0.70      0.75       771
          10       0.82      0.64      0.72       214
          11       0.72      0.71      0.71      2891
          12       0.82      0.72      0.77       421
          13       0.71      0.64      0.68      1014
          14       0.86      0.81      0.83      3133
          15       0.75      0.69      0.72      1148
          16       0.93      0.91      0.92      1326
          17       0.89    

In [60]:
print(metrics.f1_score(y_valid, predicted,average = 'macro'))

0.7456131929700762


In [62]:
prediction = svm.predict(X_test)

In [63]:
prediction

array([ 6, 20, 19, ..., 19, 26, 15], dtype=int64)

In [64]:
test_df["Category"] = prediction
baseline_file = test_df[["Id","Category"]]
baseline_file.to_csv("baseline_svm_linear.csv", index=False)

In [66]:
submissions = pd.read_csv("submission_2_r.csv")
submissions

,Id,Category
0,0,6
1,1,20
2,2,22
3,3,19
4,4,19
...,...,...
54295,54295,19
54296,54296,19
54297,54297,19
54298,54298,26


In [70]:
labels = submissions['Category']
print(labels.shape)

(54300,)


In [73]:
print(metrics.f1_score(prediction,labels,average = 'macro'))

0.8398320672605532
